In [2]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from os.path import join, exists, basename, dirname
from scipy.signal import welch

from filtering import freq_filt

In [13]:
canyon = pd.read_csv('/bsuscratch/zacharykeskinen/data/infrasound/snotel/canyon_wx.csv', comment = '#', parse_dates=['Date_Time'], index_col = ['Date_Time'])
units = canyon.iloc[0]
canyon = canyon.iloc[1:]
# convert all columns of DataFrame
canyon = canyon.apply(pd.to_numeric, errors = 'ignore')
canyon = canyon.loc[:pd.to_datetime('2022-05-15')]
canyon = canyon.tz_convert('UTC')

/tmp/ipykernel_11766/548296875.py:6: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  canyon = canyon.loc[:pd.to_datetime('2022-05-15')]


In [4]:
snotel_fp = '/bsuscratch/zacharykeskinen/data/infrasound/snotel/banner_snotel_results.csv'
snotel_fp = '/bsuscratch/zacharykeskinen/data/infrasound/snotel/banner_snotel_results.csv'
snotel = pd.read_csv(snotel_fp, comment='#', index_col=['Date'], parse_dates=['Date'])
for c in snotel.columns:
    snotel[c] = snotel[c].astype('f4')
snotel['Snow Water Equivalent'] = snotel['Snow Water Equivalent (mm) Start of Day Values']/1000
snotel['Snow Depth'] = snotel['Snow Depth (cm) Start of Day Values']/100
snotel['Average Air Temp'] = snotel['Air Temperature Average (degC)']

In [5]:
result_dir = '/bsuscratch/zacharykeskinen/data/infrasound/psd_results'
data_dir = '/bsuscratch/zacharykeskinen/data/infrasound/array_data'
with open(join(data_dir, 'merged/all_days'), 'rb') as f:
    days = pickle.load(f)
sps = 200

In [41]:
next(iter(days.items()))

('2022-01-21',
 {0.33: '/bsuscratch/zacharykeskinen/data/infrasound/array_data/be4-lower/2022-01-21_c1.parq',
  0.66: '/bsuscratch/zacharykeskinen/data/infrasound/array_data/be4-lower/2022-01-21_c2.parq',
  1: '/bsuscratch/zacharykeskinen/data/infrasound/array_data/be4-lower/2022-01-21_c3.parq',
  1.33: '/bsuscratch/zacharykeskinen/data/infrasound/array_data/a3m-upper/2022-01-21_c1.parq',
  2: '/bsuscratch/zacharykeskinen/data/infrasound/array_data/a3m-upper/2022-01-21_c3.parq',
  'snotel': Snow Water Equivalent (mm) Start of Day Values         462.0
  Snow Depth (cm) Start of Day Values                    163.0
  Precipitation Accumulation (mm) Start of Day Values    577.0
  Air Temperature Observed (degC) Start of Day Values     -2.5
  Air Temperature Maximum (degC)                           2.4
  Air Temperature Minimum (degC)                          -9.4
  Air Temperature Average (degC)                          -4.6
  Name: 2022-01-21, dtype: float32})

In [37]:
full = pd.DataFrame()
for d, fps in days.items():
    winds = canyon.loc[pd.to_datetime(d + 'T00:00:00Z'):pd.to_datetime(d + 'T23:59:00Z'), 'wind_speed_set_1']
    d = pd.to_datetime(d)
    if d < pd.to_datetime('2022-04-30'):
        sd = snotel.loc[d, 'Snow Depth (cm) Start of Day Values']/100
        try:
            for s, wind_speed in winds.iteritems():
                e = s + pd.Timedelta('1 hour')
                for h, fp in fps.items():
                    if h != 'snotel':
                        res = {}
                        res['sd_delta'] = [sd - h]
                        res['wind'] = [wind_speed]
                        arr = freq_filt(pd.read_parquet(fp)[s:e].values.ravel(), 1, kind = 'highpass')
                        power = np.sum(arr**2)/arr.size
                        res['power'] = [power]
                        df = pd.DataFrame(res)
                        full = pd.concat([full, df])
        except ValueError as e:
            print(e)

The length of the input vector x must be greater than padlen, which is 9.
{'sd_delta': [1.22], 'wind': [0.37], 'power': [3.2998620574928384e-05]}
{'sd_delta': [0.89], 'wind': [0.37], 'power': [2.775559845140385e-05]}
{'sd_delta': [0.55], 'wind': [0.37], 'power': [2.664172674207924e-05]}
{'sd_delta': [0.21999999999999997], 'wind': [0.37], 'power': [2.6548544663238784e-05]}


KeyboardInterrupt: 